In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.9 MB/s eta 0:00:00


In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
estimator = [('RF', RandomForestClassifier(n_estimators = 450, max_depth = 9)), ('XGB', XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1)),
             ('Cat', CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35)), ('LGBM', LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50)),
             ('ETC', ExtraTreesClassifier(n_estimators = 450, max_depth = 7)),('KNN', KNeighborsClassifier(n_neighbors=5)),
             ('ADB', AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50))]
Stacking = StackingClassifier( estimators=estimator, final_estimator= XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1))

# **FastText**

**Imbalanced**

In [ ]:
df_tr = pd.read_csv('/content/FT-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total metrics(FT-CV).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.947773  0.627461   
1                DecisionTreeClassifier(max_depth=5)  0.935628  0.528320   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.931579  0.467813   
3                             KNeighborsClassifier()  0.938057  0.535662   
4  <catboost.core.CatBoostClassifier object at 0x...  0.982996  0.891603   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.981377  0.884385   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.983401  0.894370   
7       LGBMClassifier(max_depth=7, random_state=50)  0.982996  0.891687   
8  StackingClassifier(estimators=[('RF',\n       ...  0.985830  0.910625   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.564978   1.000000  0.416290  0.587859     1.000000     0.416290  
1  0.499818   0.762712  0.407240  0.530973     0.987550     0.407240  
2  0.359107   1.000000  0.

**ADASYN**

In [ ]:
df_tr = pd.read_csv('/content/FT-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN(random_state=42)
xtrain, ytrain = ada.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(FT-CV(ADASYN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.987033  0.974390   
1                DecisionTreeClassifier(max_depth=5)  0.830315  0.660917   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.788732  0.636056   
3                             KNeighborsClassifier()  0.835010  0.710252   
4  <catboost.core.CatBoostClassifier object at 0x...  0.989269  0.978538   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.957299  0.915184   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.997317  0.994635   
7       LGBMClassifier(max_depth=7, random_state=50)  0.996647  0.993295   
8  StackingClassifier(estimators=[('RF',\n       ...  0.997988  0.995981   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.974062   1.000000  0.973921  0.986788     1.000000     0.973921  
1  0.660672   0.820569  0.843076  0.831670     0.817697     0.843076  
2  0.576453   0.999219  0.

**SMOTEN**

In [ ]:
df_tr = pd.read_csv('/content/FT-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import SMOTEN
smn = SMOTEN()
xtrain, ytrain = smn.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(FT-CV(SMOTEN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.977546  0.956056   
1                DecisionTreeClassifier(max_depth=5)  0.931525  0.863281   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.937750  0.882365   
3                             KNeighborsClassifier()  0.975100  0.950322   
4  <catboost.core.CatBoostClassifier object at 0x...  0.994664  0.989360   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.991774  0.983565   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.995109  0.990243   
7       LGBMClassifier(max_depth=7, random_state=50)  0.995331  0.990706   
8  StackingClassifier(estimators=[('RF',\n       ...  0.993997  0.988023   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.955091   1.000000  0.955091  0.977030     1.000000     0.955091  
1  0.863050   0.941739  0.919964  0.930724     0.943086     0.919964  
2  0.875500   1.000000  0.

**SMOTETomek**

In [ ]:
df_tr = pd.read_csv('/content/FT-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
xtrain, ytrain = smt.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("FastText-CV(SMOTETomek.csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.985549  0.971479   
1                DecisionTreeClassifier(max_depth=5)  0.805691  0.612439   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.819031  0.684467   
3                             KNeighborsClassifier()  0.841263  0.719761   
4  <catboost.core.CatBoostClassifier object at 0x...  0.991774  0.983553   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.960427  0.921326   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.996221  0.992444   
7       LGBMClassifier(max_depth=7, random_state=50)  0.996888  0.993777   
8  StackingClassifier(estimators=[('RF',\n       ...  0.998666  0.997332   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.971098   0.999543  0.971543  0.985344     0.999555     0.971543  
1  0.611383   0.824752  0.776345  0.799817     0.835038     0.776345  
2  0.638061   1.000000  0.

**NearMiss**

In [ ]:
df_tr = pd.read_csv('/content/FT-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
xtrain, ytrain = nm.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(FT-CV(NearMiss)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.984163  0.968574   
1                DecisionTreeClassifier(max_depth=5)  0.832579  0.666908   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.932127  0.871275   
3                             KNeighborsClassifier()  0.685520  0.472860   
4  <catboost.core.CatBoostClassifier object at 0x...  0.986425  0.973209   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.968326  0.936690   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.986425  0.972891   
7       LGBMClassifier(max_depth=7, random_state=50)  0.981900  0.963801   
8  StackingClassifier(estimators=[('RF',\n       ...  0.988688  0.977466   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.968326   0.995370  0.972851  0.983982     0.995475     0.972851  
1  0.665158   0.858537  0.796380  0.826291     0.868778     0.796380  
2  0.864253   0.994819  0.

**TomekLinks**

In [ ]:
df_tr = pd.read_csv('/content/FT-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks()
xtrain, ytrain = tl.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(FT-CV(TomekLinks)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.947368  0.623910   
1                DecisionTreeClassifier(max_depth=5)  0.936032  0.533442   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.931984  0.472387   
3                             KNeighborsClassifier()  0.938057  0.535662   
4  <catboost.core.CatBoostClassifier object at 0x...  0.982996  0.891603   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.981377  0.884385   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.983401  0.894370   
7       LGBMClassifier(max_depth=7, random_state=50)  0.982996  0.891687   
8  StackingClassifier(estimators=[('RF',\n       ...  0.986235  0.913151   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.560389   1.000000  0.411765  0.583333     1.000000     0.411765  
1  0.506785   0.760331  0.416290  0.538012     0.987105     0.416290  
2  0.364877   1.000000  0.

**Ind-Test**

In [ ]:
df_ts = pd.read_csv('/content/FT-Tr.csv')
columns = df_ts.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_ts[columns]
ytrain = df_ts[target]

In [ ]:
df_ts = pd.read_csv('/content/Re-FT-Ind.csv')
columns = df_ts.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtest = df_ts[columns]
ytest = df_ts[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  model.fit(xtrain, ytrain)
  pred = model.predict(xtest)
  # pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(FT-Ind).csv")

0:	learn: 0.3919275	total: 149ms	remaining: 6.58s
1:	learn: 0.2686525	total: 256ms	remaining: 5.49s
2:	learn: 0.2206487	total: 359ms	remaining: 5.03s
3:	learn: 0.1813348	total: 448ms	remaining: 4.59s
4:	learn: 0.1655488	total: 554ms	remaining: 4.43s
5:	learn: 0.1488227	total: 643ms	remaining: 4.18s
6:	learn: 0.1365108	total: 750ms	remaining: 4.07s
7:	learn: 0.1226008	total: 833ms	remaining: 3.85s
8:	learn: 0.1124686	total: 961ms	remaining: 3.85s
9:	learn: 0.0997102	total: 1.11s	remaining: 3.88s
10:	learn: 0.0915259	total: 1.26s	remaining: 3.88s
11:	learn: 0.0824956	total: 1.39s	remaining: 3.82s
12:	learn: 0.0797137	total: 1.55s	remaining: 3.81s
13:	learn: 0.0729308	total: 1.72s	remaining: 3.81s
14:	learn: 0.0626218	total: 1.88s	remaining: 3.77s
15:	learn: 0.0578231	total: 2.05s	remaining: 3.71s
16:	learn: 0.0555051	total: 2.21s	remaining: 3.64s
17:	learn: 0.0507111	total: 2.37s	remaining: 3.55s
18:	learn: 0.0472555	total: 2.54s	remaining: 3.48s
19:	learn: 0.0435461	total: 2.69s	remaini

# **LSA**

**Imbalanced**

In [ ]:
df_tr = pd.read_csv('/content/LSA-Tr.csv')

In [ ]:
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(LSA-CV).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.954288  0.683748   
1                DecisionTreeClassifier(max_depth=5)  0.958738  0.720638   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.911003  0.090590   
3                             KNeighborsClassifier()  0.963997  0.759146   
4  <catboost.core.CatBoostClassifier object at 0x...  0.995550  0.972524   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.994337  0.965039   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.993528  0.959883   
7       LGBMClassifier(max_depth=7, random_state=50)  0.995955  0.975054   
8  StackingClassifier(estimators=[('RF',\n       ...  0.996764  0.980091   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.637149   1.000000  0.490991  0.658610     1.000000     0.490991  
1  0.705797   0.894737  0.612613  0.727273     0.992889     0.612613  
2  0.016280   1.000000  0.

**ADASYN**

In [ ]:
df_tr = pd.read_csv('/content/LSA-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN()
xtrain, ytrain = ada.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(LSA-CV(ADASYN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.986444  0.973247   
1                DecisionTreeClassifier(max_depth=5)  0.900444  0.800894   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.969111  0.939916   
3                             KNeighborsClassifier()  0.997778  0.995565   
4  <catboost.core.CatBoostClassifier object at 0x...  0.998000  0.996005   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.984000  0.968153   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.997556  0.995116   
7       LGBMClassifier(max_depth=7, random_state=50)  0.998222  0.996448   
8  StackingClassifier(estimators=[('RF',\n       ...  0.999333  0.998668   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.972889   1.000000  0.972889  0.986258     1.000000     0.972889  
1  0.800889   0.901873  0.898667  0.900267     0.902222     0.898667  
2  0.938222   0.999054  0.

**SMOTEN**

In [ ]:
df_tr = pd.read_csv('/content/LSA-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import SMOTEN
smn = SMOTEN()
xtrain, ytrain = smn.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(LSA-CV(SMOTEN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.972444  0.946327   
1                DecisionTreeClassifier(max_depth=5)  0.976444  0.953492   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.954222  0.912276   
3                             KNeighborsClassifier()  0.981778  0.964196   
4  <catboost.core.CatBoostClassifier object at 0x...  0.996889  0.993797   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.996222  0.992445   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.995778  0.991584   
7       LGBMClassifier(max_depth=7, random_state=50)  0.996444  0.992914   
8  StackingClassifier(estimators=[('RF',\n       ...  0.996444  0.992914   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.944889   1.000000  0.944889  0.971664     1.000000     0.944889  
1  0.952889   0.994009  0.958667  0.976018     0.994222     0.958667  
2  0.908444   1.000000  0.

**SMOTETomek**

In [ ]:
df_tr = pd.read_csv('/content/LSA-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek()
xtrain, ytrain = smt.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(LSA-CV(SMOTETomek)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.989333  0.978889   
1                DecisionTreeClassifier(max_depth=5)  0.898667  0.797387   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.978222  0.957353   
3                             KNeighborsClassifier()  0.998889  0.997780   
4  <catboost.core.CatBoostClassifier object at 0x...  0.996889  0.993779   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.984889  0.969792   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.997111  0.994227   
7       LGBMClassifier(max_depth=7, random_state=50)  0.997556  0.995111   
8  StackingClassifier(estimators=[('RF',\n       ...  0.999556  0.999111   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.978667   1.000000  0.978667  0.989218     1.000000     0.978667  
1  0.797333   0.894112  0.904444  0.899249     0.892889     0.904444  
2  0.956444   1.000000  0.

**NearMiss**

In [ ]:
df_tr = pd.read_csv('/content/LSA-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
xtrain, ytrain = nm.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(LSA-All-CV(NearMiss)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.986486  0.973329   
1                DecisionTreeClassifier(max_depth=5)  0.966216  0.932518   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.950450  0.905357   
3                             KNeighborsClassifier()  0.909910  0.833461   
4  <catboost.core.CatBoostClassifier object at 0x...  0.988739  0.977567   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.988739  0.977567   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.977477  0.954994   
7       LGBMClassifier(max_depth=7, random_state=50)  0.986486  0.973329   
8  StackingClassifier(estimators=[('RF',\n       ...  0.990991  0.982141   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.972973   1.000000  0.972973  0.986301     1.000000     0.972973  
1  0.932432   0.972603  0.959459  0.965986     0.972973     0.959459  
2  0.900901   1.000000  0.

**TomekLinks**

In [ ]:
df_tr = pd.read_csv('/content/LSA-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks()
xtrain, ytrain = tl.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(LSA-CV(TomekLinks)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.955502  0.693534   
1                DecisionTreeClassifier(max_depth=5)  0.957524  0.712519   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.911408  0.110972   
3                             KNeighborsClassifier()  0.963997  0.759146   
4  <catboost.core.CatBoostClassifier object at 0x...  0.995550  0.972524   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.994337  0.965039   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.993528  0.959883   
7       LGBMClassifier(max_depth=7, random_state=50)  0.995955  0.975054   
8  StackingClassifier(estimators=[('RF',\n       ...  0.996359  0.977621   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.649552   1.000000  0.504505  0.670659     1.000000     0.504505  
1  0.699278   0.877419  0.612613  0.721485     0.991556     0.612613  
2  0.024330   1.000000  0.

**Ind-Test**

In [ ]:
df_tr = pd.read_csv('/content/LSA-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
df_tr = pd.read_csv('/content/Re-LSA-Ind.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtest = df_tr[columns]
ytest= df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

# cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  model.fit(xtrain, ytrain)
  pred = model.predict(xtest)
  # pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(LSA-Ind).csv")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0:	learn: 0.3618436	total: 76.3ms	remaining: 3.36s
1:	learn: 0.2419361	total: 127ms	remaining: 2.74s
2:	learn: 0.1750876	total: 163ms	remaining: 2.28s
3:	learn: 0.1439314	total: 210ms	remaining: 2.16s
4:	learn: 0.1160574	total: 263ms	remaining: 2.11s
5:	learn: 0.0971383	total: 307ms	remaining: 1.99s
6:	learn: 0.0868232	total: 362ms	remaining: 1.97s
7:	learn: 0.0705775	total: 417ms	remaining: 1.93s
8:	learn: 0.0636306	total: 473ms	remaining: 1.89s
9:	learn: 0.0575239	total: 529ms	remaining: 1.85s
10:	learn: 0.0497066	total: 589ms	remaining: 1.82s
11:	learn: 0.0447669	total: 629ms	remaining: 1.73s
12:	learn: 0.0406392	total: 681ms	remaining: 1.68s
13:	learn: 0.0378796	total: 742ms	remaining: 1.64s
14:	learn: 0.0336100	total: 795ms	remaining: 1.59s
15:	learn: 0.0308282	total: 853ms	remaining: 1.54s
16:	learn: 0.0271913	total: 910ms	remaining: 1.5s
17:	learn: 0.0249951	total: 961ms	remaining: 1.44s
18:	learn: 0.0222601	total: 1.01s	remaining: 1.38s
19:	learn: 0.0200442	total: 1.06s	remaini

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 222, number of negative: 2250
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003121 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14280
[LightGBM] [Info] Number of data points in the train set: 2472, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.089806 -> initscore=-2.316008
[LightGBM] [Info] Start training from score -2.316008
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0:	learn: 0.3618436	total: 35.2ms	remaining: 1.55s
1:	learn: 0.2419361	total: 62.7ms	remaining: 1.35s
2:	learn: 0.1750876	total: 89.5ms	remaining: 1.25s
3:	learn: 0.1439314	total: 117ms	remaining: 1.2s
4:	learn: 0.1160574	total: 143ms	remaining: 1.14s
5:	learn: 0.0971383	total: 170ms	remaining: 1.11s
6:	learn: 0.0868232	total: 196ms	remaining: 1.06s
7:	learn: 0.0705775	total: 222ms	remaining: 1.03s
8:	learn: 0.0636306	total: 252ms	remaining: 1.01s
9:	learn: 0.0575239	total: 279ms	remaining: 976ms
10:	learn: 0.0497066	total: 306ms	remaining: 946ms
11:	learn: 0.0447669	total: 332ms	remaining: 913ms
12:	learn: 0.0406392	total: 361ms	remaining: 888ms
13:	learn: 0.0378796	total: 387ms	remaining: 857ms
14:	learn: 0.0336100	total: 414ms	remaining: 828ms
15:	learn: 0.0308282	total: 441ms	remaining: 800ms
16:	learn: 0.0271913	total: 471ms	remaining: 775ms
17:	learn: 0.0249951	total: 496ms	remaining: 745ms
18:	learn: 0.0222601	total: 523ms	remaining: 716ms
19:	learn: 0.0200442	total: 556ms	remai

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **AAC**

**Imbalanced**

In [ ]:
df_tr = pd.read_csv('/content/AAC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(AAC-CV).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.974919  0.837301   
1                DecisionTreeClassifier(max_depth=5)  0.942557  0.599535   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.932443  0.479668   
3                             KNeighborsClassifier()  0.948220  0.639243   
4  <catboost.core.CatBoostClassifier object at 0x...  0.985032  0.907265   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.956311  0.715873   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.987460  0.923781   
7       LGBMClassifier(max_depth=7, random_state=50)  0.989887  0.937785   
8  StackingClassifier(estimators=[('RF',\n       ...  0.990291  0.939987   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.830019   0.959770  0.752252  0.843434     0.996889     0.752252  
1  0.584524   0.773973  0.509009  0.614130     0.985333     0.509009  
2  0.378633   0.982456  0.

**ADASYN**

In [ ]:
df_tr = pd.read_csv('/content/AAC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN()
xtrain, ytrain = ada.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(AAC-CV(ADASYN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.988999  0.978218   
1                DecisionTreeClassifier(max_depth=5)  0.861473  0.722915   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.876066  0.756998   
3                             KNeighborsClassifier()  0.954198  0.912303   
4  <catboost.core.CatBoostClassifier object at 0x...  0.989448  0.979097   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.914459  0.829004   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.991917  0.983964   
7       LGBMClassifier(max_depth=7, random_state=50)  0.989897  0.979978   
8  StackingClassifier(estimators=[('RF',\n       ...  0.996857  0.993713   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.978000   0.978676  0.999546  0.989001     0.978667     0.999546  
1  0.722894   0.862826  0.856171  0.859485     0.866667     0.856171  
2  0.751807   0.924897  0.

**SMOTEN**

In [ ]:
df_tr = pd.read_csv('/content/AAC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import SMOTEN
smn = SMOTEN()
xtrain, ytrain = smn.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(AAC-CV(SMOTEN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.990444  0.980891   
1                DecisionTreeClassifier(max_depth=5)  0.953556  0.907671   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.974222  0.948482   
3                             KNeighborsClassifier()  0.976000  0.952122   
4  <catboost.core.CatBoostClassifier object at 0x...  0.988222  0.976449   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.976667  0.953349   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.991778  0.983563   
7       LGBMClassifier(max_depth=7, random_state=50)  0.991778  0.983563   
8  StackingClassifier(estimators=[('RF',\n       ...  0.988000  0.976139   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.980889   0.991537  0.989333  0.990434     0.991556     0.989333  
1  0.907111   0.938171  0.971111  0.954357     0.936000     0.971111  
2  0.948444   0.970044  0.

**SMOTETomek**

In [ ]:
df_tr = pd.read_csv('/content/AAC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek()
xtrain, ytrain = smt.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(AAC-CV(SMOTETomek)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.985556  0.971123   
1                DecisionTreeClassifier(max_depth=5)  0.901333  0.803078   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.898667  0.802063   
3                             KNeighborsClassifier()  0.958000  0.919249   
4  <catboost.core.CatBoostClassifier object at 0x...  0.989333  0.978754   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.919556  0.839138   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.990667  0.981421   
7       LGBMClassifier(max_depth=7, random_state=50)  0.988000  0.976111   
8  StackingClassifier(estimators=[('RF',\n       ...  0.995778  0.991560   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.971111   0.983193  0.988000  0.985591     0.983111     0.988000  
1  0.802667   0.888889  0.917333  0.902887     0.885333     0.917333  
2  0.797333   0.947159  0.

**NearMiss**

In [ ]:
df_tr = pd.read_csv('/content/AAC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
xtrain, ytrain = nm.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(AAC-CV(NearMiss)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.950450  0.900937   
1                DecisionTreeClassifier(max_depth=5)  0.921171  0.843378   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.916667  0.835787   
3                             KNeighborsClassifier()  0.826577  0.659616   
4  <catboost.core.CatBoostClassifier object at 0x...  0.954955  0.909910   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.934685  0.871930   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.943694  0.888913   
7       LGBMClassifier(max_depth=7, random_state=50)  0.959459  0.918956   
8  StackingClassifier(estimators=[('RF',\n       ...  0.961712  0.924559   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.900901   0.946429  0.954955  0.950673     0.945946     0.954955  
1  0.842342   0.943128  0.896396  0.919169     0.945946     0.896396  
2  0.833333   0.951220  0.

**TomekLinks**

In [ ]:
df_tr = pd.read_csv('/content/AAC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks()
xtrain, ytrain = tl.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(AAC-CV(TomekLinks)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.973706  0.828727   
1                DecisionTreeClassifier(max_depth=5)  0.943366  0.604509   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.932039  0.475177   
3                             KNeighborsClassifier()  0.948220  0.639243   
4  <catboost.core.CatBoostClassifier object at 0x...  0.985032  0.907265   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.956311  0.715873   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.987460  0.923781   
7       LGBMClassifier(max_depth=7, random_state=50)  0.989887  0.937785   
8  StackingClassifier(estimators=[('RF',\n       ...  0.990291  0.939987   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.818944   0.970060  0.729730  0.832905     0.997778     0.729730  
1  0.588400   0.784722  0.509009  0.617486     0.986222     0.509009  
2  0.372998   0.982143  0.

**Ind-Test**

In [ ]:
df_tr = pd.read_csv('/content/AAC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
df_tr = pd.read_csv('/content/AAC-Ind.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtest = df_tr[columns]
ytest = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

# cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  model.fit(xtrain, ytrain)
  pred = model.predict(xtest)
  # pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(AAC-Ind).csv")

0:	learn: 0.3224236	total: 71.2ms	remaining: 3.13s
1:	learn: 0.2005744	total: 90.9ms	remaining: 1.95s
2:	learn: 0.1585602	total: 110ms	remaining: 1.54s
3:	learn: 0.1292872	total: 130ms	remaining: 1.33s
4:	learn: 0.1160448	total: 150ms	remaining: 1.2s
5:	learn: 0.1088562	total: 171ms	remaining: 1.11s
6:	learn: 0.0999704	total: 189ms	remaining: 1.03s
7:	learn: 0.0913168	total: 209ms	remaining: 965ms
8:	learn: 0.0864968	total: 229ms	remaining: 915ms
9:	learn: 0.0826496	total: 248ms	remaining: 870ms
10:	learn: 0.0755345	total: 265ms	remaining: 820ms
11:	learn: 0.0741817	total: 282ms	remaining: 774ms
12:	learn: 0.0669797	total: 299ms	remaining: 737ms
13:	learn: 0.0605790	total: 316ms	remaining: 700ms
14:	learn: 0.0589727	total: 335ms	remaining: 670ms
15:	learn: 0.0538223	total: 354ms	remaining: 642ms
16:	learn: 0.0506963	total: 373ms	remaining: 614ms
17:	learn: 0.0468650	total: 389ms	remaining: 583ms
18:	learn: 0.0442445	total: 399ms	remaining: 547ms
19:	learn: 0.0407114	total: 410ms	remain

# **CTDC**

**Imbalanced**

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(CTDC-CV).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.967233  0.783627   
1                DecisionTreeClassifier(max_depth=5)  0.953074  0.679408   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.955097  0.689450   
3                             KNeighborsClassifier()  0.948220  0.645854   
4  <catboost.core.CatBoostClassifier object at 0x...  0.985032  0.907577   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.960761  0.745603   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.987864  0.926077   
7       LGBMClassifier(max_depth=7, random_state=50)  0.987864  0.924955   
8  StackingClassifier(estimators=[('RF',\n       ...  0.988269  0.926666   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.774376   0.922156  0.693694  0.791774     0.994222     0.693694  
1  0.665416   0.848684  0.581081  0.689840     0.989778     0.581081  
2  0.654493   0.958678  0.

**ADASYN**

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN()
xtrain, ytrain = ada.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(CTDC-CV(ADASYN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.974745  0.949608   
1                DecisionTreeClassifier(max_depth=5)  0.807045  0.614906   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.844705  0.701414   
3                             KNeighborsClassifier()  0.940408  0.887098   
4  <catboost.core.CatBoostClassifier object at 0x...  0.985600  0.971577   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.939521  0.880014   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.989809  0.979757   
7       LGBMClassifier(max_depth=7, random_state=50)  0.988259  0.976710   
8  StackingClassifier(estimators=[('RF',\n       ...  0.996899  0.993811   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.949487   0.967391  0.982774  0.975022     0.966667     0.982774  
1  0.614022   0.792034  0.834364  0.812648     0.779556     0.834364  
2  0.689584   0.922204  0.

**SMOTEN**

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import SMOTEN
smn = SMOTEN()
xtrain, ytrain = smn.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(CTDC-CV(SMOTEN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.988444  0.976965   
1                DecisionTreeClassifier(max_depth=5)  0.948444  0.896902   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.966000  0.933017   
3                             KNeighborsClassifier()  0.976667  0.953541   
4  <catboost.core.CatBoostClassifier object at 0x...  0.990000  0.980001   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.982889  0.965799   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.992667  0.985333   
7       LGBMClassifier(max_depth=7, random_state=50)  0.993333  0.986667   
8  StackingClassifier(estimators=[('RF',\n       ...  0.978889  0.958495   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.976889   0.994599  0.982222  0.988372     0.994667     0.982222  
1  0.896889   0.950849  0.945778  0.948307     0.951111     0.945778  
2  0.932000   0.988811  0.

**SMOTETomek**

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek()
xtrain, ytrain = smt.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(CTDC-CV(SMOTETomek)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.976667  0.953436   
1                DecisionTreeClassifier(max_depth=5)  0.840000  0.680017   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.867111  0.750065   
3                             KNeighborsClassifier()  0.947778  0.900230   
4  <catboost.core.CatBoostClassifier object at 0x...  0.985556  0.971242   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.947111  0.894240   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.985778  0.971631   
7       LGBMClassifier(max_depth=7, random_state=50)  0.988000  0.976031   
8  StackingClassifier(estimators=[('RF',\n       ...  0.990444  0.980891   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.953333   0.983762  0.969333  0.976494     0.984000     0.969333  
1  0.680000   0.842435  0.836444  0.839429     0.843556     0.836444  
2  0.734222   0.961453  0.

**NearMiss**

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
xtrain, ytrain = nm.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(CTDC-CV(NearMiss)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.936937  0.875617   
1                DecisionTreeClassifier(max_depth=5)  0.869369  0.741755   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.905405  0.821687   
3                             KNeighborsClassifier()  0.869369  0.743094   
4  <catboost.core.CatBoostClassifier object at 0x...  0.961712  0.923508   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.943694  0.887396   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.952703  0.905415   
7       LGBMClassifier(max_depth=7, random_state=50)  0.963964  0.928079   
8  StackingClassifier(estimators=[('RF',\n       ...  0.961712  0.924559   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.873874   0.966346  0.905405  0.934884     0.968468     0.905405  
1  0.738739   0.905941  0.824324  0.863208     0.914414     0.824324  
2  0.810811   0.983871  0.

**TomekLinks**

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks()
xtrain, ytrain = tl.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(CTDC-CV(TomekLinks)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.966828  0.780956   
1                DecisionTreeClassifier(max_depth=5)  0.953074  0.680076   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.954288  0.682944   
3                             KNeighborsClassifier()  0.948220  0.645854   
4  <catboost.core.CatBoostClassifier object at 0x...  0.985032  0.907577   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.960761  0.745603   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.987864  0.926077   
7       LGBMClassifier(max_depth=7, random_state=50)  0.987864  0.924955   
8  StackingClassifier(estimators=[('RF',\n       ...  0.987864  0.924182   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.772112   0.916667  0.693694  0.789744     0.993778     0.693694  
1  0.666992   0.844156  0.585586  0.691489     0.989333     0.585586  
2  0.644638   0.965812  0.

**Ind-Test**

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
df_tr = pd.read_csv('/content/CTDC-Ind.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtest = df_tr[columns]
ytest = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

# cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  model.fit(xtrain, ytrain)
  pred = model.predict(xtest)
  # pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(CTDC-Ind).csv")

0:	learn: 0.3586288	total: 101ms	remaining: 4.44s
1:	learn: 0.2387314	total: 133ms	remaining: 2.86s
2:	learn: 0.1838333	total: 172ms	remaining: 2.41s
3:	learn: 0.1574430	total: 209ms	remaining: 2.14s
4:	learn: 0.1386140	total: 248ms	remaining: 1.99s
5:	learn: 0.1305302	total: 289ms	remaining: 1.88s
6:	learn: 0.1233358	total: 323ms	remaining: 1.75s
7:	learn: 0.1182270	total: 359ms	remaining: 1.66s
8:	learn: 0.1038264	total: 399ms	remaining: 1.6s
9:	learn: 0.0897411	total: 430ms	remaining: 1.5s
10:	learn: 0.0825722	total: 454ms	remaining: 1.4s
11:	learn: 0.0725579	total: 479ms	remaining: 1.32s
12:	learn: 0.0647758	total: 503ms	remaining: 1.24s
13:	learn: 0.0583320	total: 533ms	remaining: 1.18s
14:	learn: 0.0535854	total: 559ms	remaining: 1.12s
15:	learn: 0.0496486	total: 599ms	remaining: 1.09s
16:	learn: 0.0474797	total: 638ms	remaining: 1.05s
17:	learn: 0.0420768	total: 680ms	remaining: 1.02s
18:	learn: 0.0381864	total: 719ms	remaining: 983ms
19:	learn: 0.0366009	total: 777ms	remaining:

# **GDPC**

**Imbalanced**

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(GDPC-CV).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.961974  0.744471   
1                DecisionTreeClassifier(max_depth=5)  0.934871  0.537537   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.927994  0.427229   
3                             KNeighborsClassifier()  0.937298  0.579366   
4  <catboost.core.CatBoostClassifier object at 0x...  0.980987  0.882159   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.954288  0.701120   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.983819  0.901432   
7       LGBMClassifier(max_depth=7, random_state=50)  0.984628  0.906361   
8  StackingClassifier(estimators=[('RF',\n       ...  0.985437  0.909633   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.730149   0.915584  0.635135  0.750000     0.994222     0.635135  
1  0.520882   0.719424  0.450450  0.554017     0.982667     0.450450  
2  0.327439   0.923077  0.

**ADASYN**

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN()
xtrain, ytrain = ada.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(GDPC-CV(ADASYN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.981531  0.963418   
1                DecisionTreeClassifier(max_depth=5)  0.820427  0.661764   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.869826  0.741879   
3                             KNeighborsClassifier()  0.930797  0.869989   
4  <catboost.core.CatBoostClassifier object at 0x...  0.988652  0.977466   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.922563  0.845974   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.990209  0.980518   
7       LGBMClassifier(max_depth=7, random_state=50)  0.990877  0.981847   
8  StackingClassifier(estimators=[('RF',\n       ...  0.993992  0.987996   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.963063   0.968764  0.995098  0.981754     0.968000     0.995098  
1  0.640973   0.756333  0.944742  0.840103     0.696444     0.944742  
2  0.739625   0.900918  0.

**SMOTEN**

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.over_sampling import SMOTEN
smn = SMOTEN()
xtrain, ytrain = smn.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(GDPC-CV(SMOTEN)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.984667  0.969345   
1                DecisionTreeClassifier(max_depth=5)  0.945778  0.891832   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.948889  0.897813   
3                             KNeighborsClassifier()  0.964667  0.929341   
4  <catboost.core.CatBoostClassifier object at 0x...  0.986222  0.972445   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.973778  0.947580   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.989556  0.979116   
7       LGBMClassifier(max_depth=7, random_state=50)  0.991111  0.982224   
8  StackingClassifier(estimators=[('RF',\n       ...  0.975556  0.951807   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.969333   0.982309  0.987111  0.984704     0.982222     0.987111  
1  0.891556   0.934952  0.958222  0.946444     0.933333     0.958222  
2  0.897778   0.944934  0.

**SMOTETomek**

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek()
xtrain, ytrain = smt.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(GDPC-CV(SMOTETomek)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.977333  0.954685   
1                DecisionTreeClassifier(max_depth=5)  0.834222  0.668482   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.892000  0.786635   
3                             KNeighborsClassifier()  0.937333  0.881619   
4  <catboost.core.CatBoostClassifier object at 0x...  0.987778  0.975687   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.928667  0.857649   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.991111  0.982310   
7       LGBMClassifier(max_depth=7, random_state=50)  0.990222  0.980510   
8  StackingClassifier(estimators=[('RF',\n       ...  0.993333  0.986681   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.954667   0.974382  0.980444  0.977404     0.974222     0.980444  
1  0.668444   0.837826  0.828889  0.833333     0.839556     0.828889  
2  0.784000   0.926912  0.

**NearMiss**

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
xtrain, ytrain = nm.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(GDPC-CV(NearMiss)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.912162  0.824400   
1                DecisionTreeClassifier(max_depth=5)  0.885135  0.770904   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.891892  0.783911   
3                             KNeighborsClassifier()  0.795045  0.593283   
4  <catboost.core.CatBoostClassifier object at 0x...  0.909910  0.820353   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.914414  0.829132   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.907658  0.815986   
7       LGBMClassifier(max_depth=7, random_state=50)  0.909910  0.820353   
8  StackingClassifier(estimators=[('RF',\n       ...  0.914414  0.829368   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.824324   0.906667  0.918919  0.912752     0.905405     0.918919  
1  0.770270   0.901408  0.864865  0.882759     0.905405     0.864865  
2  0.783784   0.899083  0.

**TomekLinks**

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks()
xtrain, ytrain = tl.fit_resample(xtrain, ytrain)

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(GDPC-CV(TomekLinks)).csv")

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=9, n_estimato...  0.962379  0.747538   
1                DecisionTreeClassifier(max_depth=5)  0.934466  0.535179   
2  ExtraTreesClassifier(max_depth=7, n_estimators...  0.927994  0.427229   
3                             KNeighborsClassifier()  0.937298  0.579366   
4  <catboost.core.CatBoostClassifier object at 0x...  0.980987  0.882159   
5  AdaBoostClassifier(learning_rate=0.5, n_estima...  0.954288  0.701120   
6  XGBClassifier(base_score=0.5, booster=None, ca...  0.983819  0.901432   
7       LGBMClassifier(max_depth=7, random_state=50)  0.984628  0.906361   
8  StackingClassifier(estimators=[('RF',\n       ...  0.984628  0.904913   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.733647   0.916129  0.639640  0.753316     0.994222     0.639640  
1  0.519080   0.714286  0.450450  0.552486     0.982222     0.450450  
2  0.327439   0.923077  0.

**Ind-Test**

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Tr.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtrain = df_tr[columns]
ytrain = df_tr[target]

In [ ]:
df_tr = pd.read_csv('/content/GDPC-Ind.csv')
columns = df_tr.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Target"]]
# Store the variable we are predicting
target = "Target"
xtest= df_tr[columns]
ytest = df_tr[target]

In [ ]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

# cv = KFold(n_splits=5, random_state=1, shuffle=True)

# create model
models = [RandomForestClassifier(n_estimators = 450, max_depth = 9),
          DecisionTreeClassifier(max_depth = 5),
          ExtraTreesClassifier(n_estimators = 450, max_depth = 7),
          KNeighborsClassifier(n_neighbors=5),
          CatBoostClassifier(depth= 7, iterations = 45, learning_rate = 0.35),
          AdaBoostClassifier(n_estimators = 350, learning_rate = 0.5, random_state = 50),
          XGBClassifier(n_estimators = 350,max_depth = 7, base_score = 0.5, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 7,random_state = 50),
          Stacking]
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  model.fit(xtrain, ytrain)
  pred = model.predict(xtest)
  # pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytest, pred)
  mcc = matthews_corrcoef(ytest, pred)
  cm1 = confusion_matrix(ytest, pred)
  kappa = cohen_kappa_score(ytest, pred)
  f1 = f1_score(ytest, pred)
  precision_score = precision_score(ytest, pred)
  recall_score = recall_score(ytest, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)
total_Metics.to_csv("total_Metics(GDPC-Ind).csv")

0:	learn: 0.3375109	total: 77.6ms	remaining: 3.42s
1:	learn: 0.2176891	total: 99.8ms	remaining: 2.15s
2:	learn: 0.1674932	total: 119ms	remaining: 1.67s
3:	learn: 0.1458531	total: 138ms	remaining: 1.42s
4:	learn: 0.1277541	total: 159ms	remaining: 1.27s
5:	learn: 0.1210375	total: 176ms	remaining: 1.15s
6:	learn: 0.1151019	total: 195ms	remaining: 1.06s
7:	learn: 0.1081074	total: 218ms	remaining: 1.01s
8:	learn: 0.1020136	total: 238ms	remaining: 952ms
9:	learn: 0.0970428	total: 261ms	remaining: 913ms
10:	learn: 0.0860568	total: 289ms	remaining: 894ms
11:	learn: 0.0795166	total: 311ms	remaining: 854ms
12:	learn: 0.0741892	total: 331ms	remaining: 815ms
13:	learn: 0.0665348	total: 352ms	remaining: 780ms
14:	learn: 0.0634794	total: 374ms	remaining: 748ms
15:	learn: 0.0622037	total: 397ms	remaining: 720ms
16:	learn: 0.0590992	total: 419ms	remaining: 691ms
17:	learn: 0.0569823	total: 439ms	remaining: 659ms
18:	learn: 0.0554606	total: 454ms	remaining: 621ms
19:	learn: 0.0531405	total: 471ms	remai